<a href="https://colab.research.google.com/github/Tom-Lennox/Kaggle/blob/master/porto_seguro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 【行ったこと】

# ▼ 

In [6]:
# kaggle APIセット
!pip install kaggle

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
# 「Download 100%.」と表示で成功。

# [kaggle.json]を持参する。
from google.colab import drive
drive.mount('/content/drive')

# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
# titanicをダウンロードします。
!kaggle competitions download -c porto-seguro-safe-driver-prediction

# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
# titanicをダウンロードします。
!kaggle competitions download -c porto-seguro-safe-driver-prediction
!ls

# ▼ unzip
!unzip '*.zip'
!ls
# ▲ unzip

# ▼ kaggle notebookの場合
# # !cd ../
# !ls ../input/porto-seguro-safe-driver-prediction

# TRAIN_DATA = '../input/porto-seguro-safe-driver-predicti1on/train.csv'
# df = pd.read_csv(TRAIN_DATA)
# ▲ kaggle notebookの場合

# ▼ 行表示
# ctrl + m + l
# ▲ 行表示

Download 100%.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 83% 25.0M/30.0M [00:00<00:00, 47.7MB/s]
100% 30.0M/30.0M [00:00<00:00, 75.1MB/s]
  0% 0.00/2.12M [00:00<?, ?B/s]
100% 2.12M/2.12M [00:00<00:00, 143MB/s]
 92% 41.0M/44.4M [00:00<00:00, 32.2MB/s]
100% 44.4M/44.4M [00:00<00:00, 81.5MB/s]
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
adc.json  sample_data		     test.csv.zip
drive	  sample_submission.csv.zip  train.csv.zip


In [26]:
import pandas as pd
import numpy as np

from logging import getLogger

TRAIN_DATA = './train.csv'
TEST_DATA =  './test.csv'

logger = getLogger(__name__)

def read_csv(path):
  logger.debug('enter')
  logger.debug('exit')
  return df

def load_train_data():
  logger.debug('enter')
  df = pd.read_csv(TRAIN_DATA)
  logger.debug('exit')
  return df

def load_test_data():
  logger.debug('enter')
  df = pd.read_csv(TEST_DATA)
  logger.debug('exit')
  return df

if __name__ == '__main__':
  print(load_train_data().head())
  print(load_test_data().head())
  print(load_train_data().info())
  print(load_test_data().info())

   id  target  ps_ind_01  ...  ps_calc_18_bin  ps_calc_19_bin  ps_calc_20_bin
0   7       0          2  ...               0               0               1
1   9       0          1  ...               0               1               0
2  13       0          5  ...               0               1               0
3  16       0          0  ...               0               0               0
4  17       0          0  ...               1               1               0

[5 rows x 59 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595212 entries, 0 to 595211
Data columns (total 59 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              595212 non-null  int64  
 1   target          595212 non-null  int64  
 2   ps_ind_01       595212 non-null  int64  
 3   ps_ind_02_cat   595212 non-null  int64  
 4   ps_ind_03       595212 non-null  int64  
 5   ps_ind_04_cat   595212 non-null  int64  
 6   ps_ind_05_cat   595212 no

In [45]:
from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger
from sklearn.linear_model import LogisticRegression
!mkdir result_tmp
# from load_data import load_train_data

DIR = 'result_tmp/'
SAMPLE_SUBMIT_FILE = './sample_submission.csv'

# 結果出力先

if __name__ == '__main__':
  log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s][%(funcName)s] %(message)s ')
  handler = StreamHandler()
  handler.setLevel('INFO')
  handler.setFormatter(log_fmt)
  logger.addHandler(handler)
  # logの出力は任意、StreamHandler()を流す
  # 標準出力：INFO　Level 以上
  # ファイル出力：debug level 以上

  handler = FileHandler(DIR + 'train.py.log', 'a')
  handler.setLevel(DEBUG)
  handler.setFormatter(log_fmt)
  logger.setLevel(DEBUG)
  logger.addHandler(handler)

  logger.info('start')
  #logを取るメリット：セクション間の実行時間がわかる。」
  df = load_train_data()

  x_train = df.drop('target', axis=1)
  # targetはdataに入ると過学習になる。
  y_train = df['target'].values
  # valuesにするとpdからnp配列になる。
  use_cols = x_train.columns.values
  # 並び順固定
  logger.debug('train columns: {} {}'.format(use_cols.shape, use_cols))
  logger.info('data preparation end {}'.format(x_train.shape))

  clf = LogisticRegression(random_state=0)
  clf.fit(x_train, y_train)

  logger.info('train end')

  df = load_test_data()

  x_test = df[use_cols].sort_values('id')
  # + 念の為sort
  logger.info('test data load end {}'.format(x_test.shape))
  pred_test = clf.predict_proba(x_test)

  df_submit = pd.read_csv(SAMPLE_SUBMIT_FILE).sort_values('id')
  df_submit['target'] = pred_test

  df_submit.to_csv(DIR + 'submit.csv')
  # use_cols = x_train.columns.values

  # logger.debug('train columns: {} {}'.format(use_cols.shape, use_cols))

  # logger.info('data preparation end {}'.format(x_train.shape))

  # cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
  # all_params = {'C': [10**i for i in range(-1, 2)],
  #               'fit_intercept': [True, False],
  #               'penalty': ['l2', 'l1'],
  #               'random_state': [0]}
  # min_score = 100
  # min_params = None

mkdir: cannot create directory ‘result_tmp’: File exists


2020-05-17 14:09:00,132 __main__ 27 [INFO][<module>] start 
2020-05-17 14:09:00,132 __main__ 27 [INFO][<module>] start 
2020-05-17 14:09:00,132 __main__ 27 [INFO][<module>] start 
2020-05-17 14:09:00,132 __main__ 27 [INFO][<module>] start 
2020-05-17 14:09:00,132 __main__ 27 [INFO][<module>] start 
2020-05-17 14:09:00,132 __main__ 27 [INFO][<module>] start 
2020-05-17 14:09:00,132 __main__ 27 [INFO][<module>] start 
2020-05-17 14:09:00,132 __main__ 27 [INFO][<module>] start 
2020-05-17 14:09:00,132 __main__ 27 [INFO][<module>] start 
2020-05-17 14:09:02,235 __main__ 38 [INFO][<module>] data preparation end (595212, 58) 
2020-05-17 14:09:02,235 __main__ 38 [INFO][<module>] data preparation end (595212, 58) 
2020-05-17 14:09:02,235 __main__ 38 [INFO][<module>] data preparation end (595212, 58) 
2020-05-17 14:09:02,235 __main__ 38 [INFO][<module>] data preparation end (595212, 58) 
2020-05-17 14:09:02,235 __main__ 38 [INFO][<module>] data preparation end (595212, 58) 
2020-05-17 14:09:02,

In [42]:
!ls

adc.json    sample_data		       test.csv      train.csv.zip
drive	    sample_submission.csv      test.csv.zip  train.py.log
result_tmp  sample_submission.csv.zip  train.csv
